In [1]:
from __future__ import division
from __future__ import print_function

import datetime
import json
import logging
import os
import pickle
import time

import numpy as np
import HTorch.optimizers as optimizers
import HTorch
import torch
from train_config import parser
from models.base_models import NCModel, LPModel
from data_utils import load_data
from train_utils import get_dir_name, format_metrics

# for nc
args_str = "--c 1.0 --task nc --dataset disease_nc --model HGCN --dim 16 --lr 0.01 --dim 16 --num-layers 2 --act relu --bias 1 --dropout 0 --weight-decay 0 --manifold PoincareBall --log-freq 5 --cuda 0"
# for lp
#args_str = "--task lp --dataset cora --model HGCN --lr 0.01 --dim 16 --num-layers 2 --act relu --bias 1 --dropout 0.5 --weight-decay 0.001 --manifold PoincareBall --log-freq 5 --cuda 0 --c None"
#
args = parser.parse_args(args_str.split())

In [2]:
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if int(args.double_precision):
    torch.set_default_dtype(torch.float64)
if int(args.cuda) >= 0:
    torch.cuda.manual_seed(args.seed)
args.device = 'cuda:' + str(args.cuda) if int(args.cuda) >= 0 else 'cpu'
args.patience = args.epochs if not args.patience else  int(args.patience)
logging.getLogger().setLevel(logging.INFO)

logging.info(f'Using: {args.device}')
logging.info("Using seed {}.".format(args.seed))

# Load data
data = load_data(args, os.path.join('./data', args.dataset))
args.n_nodes, args.feat_dim = data['features'].shape

INFO:root:Using: cuda:0
INFO:root:Using seed 1234.


In [3]:
if args.task == 'nc':
    Model = NCModel
    args.n_classes = int(data['labels'].max() + 1)
    logging.info(f'Num classes: {args.n_classes}')
else:
    args.nb_false_edges = len(data['train_edges_false'])
    args.nb_edges = len(data['train_edges'])
    if args.task == 'lp':
        Model = LPModel
    else:
        #Model = RECModel
        # No validation for reconstruction task
        args.eval_freq = args.epochs + 1

INFO:root:Num classes: 2


In [6]:
from HTorch.layers.layers import *
from HTorch import HTensor
import models.encoders as encoders
from models.encoders import *

class HGCN(Encoder):
    """
    Hyperbolic-GCN.
    """

    def __init__(self, c, args):
        super(HGCN, self).__init__(c)
        self.manifold = HTorch.get_manifold(args.manifold)()
        assert args.num_layers > 1
        dims, acts, self.curvatures = get_dim_act_curv(args)
        self.curvatures.append(self.c)
        hgc_layers = []
         
        for i in range(len(dims) - 1):
            c_in, c_out = self.curvatures[i], self.curvatures[i + 1]
            #print(c_in, c_out)
            in_dim, out_dim = dims[i], dims[i + 1]
            #print(in_dim, out_dim， args.bias)
            act = acts[i]
            #print(in_dim, out_dim, args.bias, args.local_agg, args.use_att)

            hgc_layers.append(
                    HyperbolicGraphConvolution(
                            in_dim, out_dim, c_in, c_out, args.dropout, act, args.bias, args.use_att, args.local_agg
                    )
            )
        self.layers = nn.Sequential(*hgc_layers)
        self.encode_graph = True

    def encode(self, x, adj):
        
        x_tan = self.manifold.proj_tan0(x, c=self.curvatures[0])
        x_hyp = self.manifold.expmap0(x_tan, c=self.curvatures[0])
        x_hyp = self.manifold.proj(x_hyp, c=self.curvatures[0])
    
        return super(HGCN, self).encode(x_hyp, adj)
    
encoder = HGCN(args.c, args).cuda()
# fix encode() in base_models.py
def encode(x, adj, c, manifold_name, encoder):
    if manifold_name == 'Lorentz':
        o = torch.zeros_like(x)
        x = torch.cat([o[:, 0:1], x], dim=1)
    x = HTensor(x, manifold=manifold_name, curvature=c)
    print(x)
    #adj = HTensor(adj, manifold=manifold_name, curvature=c)
    h = encoder.encode(x, adj)
    
    return

encode(data['features'].cuda(), data['adj_train_norm'].cuda(), args.c, args.manifold, encoder)
encoder.encode(HTensor(data['features'].cuda(), manifold=args.manifold, curvature=args.c), data['adj_train_norm'].cuda())
# encoder.encode(data['features'].cuda(), data['adj_train_norm'].cuda())

data['features']

Hyperbolic HTensor([[-2.2495e-04,  4.5936e-03, -2.0315e-04,  ...,  1.7924e-03,
          -4.9372e-03,  2.7925e-03],
         [ 6.4418e-04,  2.3200e-03,  1.6557e-03,  ...,  7.7525e-05,
          -2.0203e-03,  7.3659e-05],
         [-2.8468e-03,  4.9286e-03, -7.9001e-04,  ...,  3.4674e-03,
          -5.9888e-04,  4.9276e-03],
         ...,
         [ 1.9970e-05,  1.1449e-03, -1.5220e-03,  ...,  1.7797e-03,
           1.5051e-03, -2.0007e-03],
         [-6.3209e-04,  6.1707e-03,  1.4591e-03,  ...,  3.2707e-03,
          -5.8705e-04,  7.6107e-04],
         [ 1.0904e-03,  6.6974e-04,  7.8708e-04,  ...,  2.7830e-04,
           1.3530e-03, -1.8505e-03]], device='cuda:0'), manifold=PoincareBall, curvature=1.0
!!!!! x type of Hyperbolic HTensor([[-2.2448e-04,  4.5839e-03, -2.0272e-04,  ...,  1.7887e-03,
          -4.9268e-03,  2.7866e-03],
         [ 6.4313e-04,  2.3162e-03,  1.6531e-03,  ...,  7.7399e-05,
          -2.0170e-03,  7.3540e-05],
         [-2.8410e-03,  4.9186e-03, -7.8839e-04,  ..

tensor([[-2.2495e-04,  4.5936e-03, -2.0315e-04,  ...,  1.7924e-03,
         -4.9372e-03,  2.7925e-03],
        [ 6.4418e-04,  2.3200e-03,  1.6557e-03,  ...,  7.7525e-05,
         -2.0203e-03,  7.3659e-05],
        [-2.8468e-03,  4.9286e-03, -7.9001e-04,  ...,  3.4674e-03,
         -5.9888e-04,  4.9276e-03],
        ...,
        [ 1.9970e-05,  1.1449e-03, -1.5220e-03,  ...,  1.7797e-03,
          1.5051e-03, -2.0007e-03],
        [-6.3209e-04,  6.1707e-03,  1.4591e-03,  ...,  3.2707e-03,
         -5.8705e-04,  7.6107e-04],
        [ 1.0904e-03,  6.6974e-04,  7.8708e-04,  ...,  2.7830e-04,
          1.3530e-03, -1.8505e-03]])